#**Importing required libraries**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

In [105]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from keras.utils import pad_sequences
from keras.layers import Dropout


In [4]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('wordnet')
  
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#@title


#**Importing the dataset**

In [7]:
main_file = pd.read_csv("/content/drive/MyDrive/colab files/training.1600000.processed.noemoticon.csv",encoding = 'latin-1')

In [8]:
main_file.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
main_file.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [10]:
main_file.describe()

,0,1467810369
count,1.599999e+06,1.599999e+06
mean,2.000001e+00,1.998818e+09
std,2.000001e+00,1.935757e+08
min,0.000000e+00,1.467811e+09
25%,0.000000e+00,1.956916e+09
50%,4.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


###checking column names

In [11]:
main_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

###confirming the import and column name

In [12]:
tweet_column = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
main_file[tweet_column][0]

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [13]:
main_file[tweet_column].shape

(1599999,)

###Finding the average number of characters in each tweet

In [14]:
main_file[tweet_column].apply(len).mean()

74.09008568130355

In [15]:
tweets = main_file[tweet_column]

###checking for null values in the dataset

In [16]:
pd.isnull(tweets).sum() #i.e no null values

0

# Data Pre-processing

###**defininf removing punctuations**

In [56]:
def remove_punctuation(sentence):

  no_punctuation = re.sub(r'[^\w\s]'+'@'+'',"", str(sentence))

  return no_punctuation

In [57]:
remove_punctuation(tweets[1])

'@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'

###**Defining stop words remover**

In [46]:
def remove_stopwords(sentence):

  stop_words = set(stopwords.words('english'))

  #word_tokens = word_tokenize(sentence)
  word_tokens = sentence

  no_stopwords_tokens = ""
  
  for w in word_tokens:
    if not w.lower() in stop_words:
      no_stopwords_tokens = no_stopwords_tokens + str(w.lower())

  return no_stopwords_tokens

###**Defining Lemmatizing tokens**

In [20]:
def tokens_lemmatizer(tokens):

  lemmatizer = WordNetLemmatizer()

  lemmatized_tokens = [lemmatizer.lemmatize(w) for w in tokens]

  return lemmatized_tokens
  


###**Defining max_lenght to find the highest lenght of tweets**



In [21]:
def pad_sequences(corpus):

  max_corpus_length = max([len(x) for x in corpus])

  corpus = np.array(pad_sequences(corpus, maxlen=max_corpus_length, padding='post'))

  return corpus

###**Cleaning tweets**

In [ ]:
clean_tweets = []

for x in tweets:

  clean_tweets_process = []

  clean_tweets_process = tokens_lemmatizer(remove_punctuation(x))

  clean_tweets.append(clean_tweets_process)



###**Maping characters of the corpus to numbers**

In [93]:
text = ""
corpus = []

for x in clean_tweets:
  for y in x:
    text = text + str(y).lower()
    corpus.append(y)

chars = sorted(list(set(text.lower())))

char_to_int = dict((c, i) for i, c in enumerate(chars))

In [204]:
len(corpus)

373659

In [94]:
n_chars = len(text)
print(n_chars)

373659


In [95]:
n_vocab = len(chars)
print(n_vocab)

40


In [96]:
max([len(x) for x in clean_tweets])

144

In [97]:
max([len(x) for x in tweets])

374

###**Encoding the input and output pairs**

In [106]:
# preparing for input to output pairs encoded as integers
sequence_length = 144
dataX = []
dataY = []
for i in range(0, n_chars - sequence_length, 1):
 seq_in = text[i:i + sequence_length]
 seq_out = text[i + sequence_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  373515


###**Normalizing and Reshaping the data**

In [107]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, sequence_length, 1))

# normalizing values to 0-1
X = X / float(n_vocab)

#hot encoding
y = to_categorical(dataY)

In [152]:
X.shape

(373515, 144, 1)

In [108]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

#Model Building

###**Defining the model**

In [117]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

###**Creating Checkpoints**

In [114]:
# creating checkpoints
filepath="/content/drive/MyDrive/colab files/Tweet generator project/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

#Training the Model

In [120]:
# fitting the model
model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list)


2919/2919 [==============================] - ETA: 0s - loss: 2.4246
Epoch 1: loss improved from 2.70897 to 2.42464, saving model to /content/drive/MyDrive/colab files/Tweet generator project/weights-improvement-01-2.4246.hdf5
2919/2919 [==============================] - 7898s 3s/step - loss: 2.4246


In [121]:
model.fit(X,y, epochs = 1, batch_size = 128, callbacks = callbacks_list)

2919/2919 [==============================] - ETA: 0s - loss: 2.2693
Epoch 1: loss improved from 2.42464 to 2.26928, saving model to /content/drive/MyDrive/colab files/Tweet generator project/weights-improvement-01-2.2693.hdf5
2919/2919 [==============================] - 7592s 3s/step - loss: 2.2693


###**Reverse mapping the integer**

In [122]:
# reverse mapping int to characters

int_to_char = dict((i, c) for i, c in enumerate(chars))

#Text Generation

###**Defining a function to generate text**

In [195]:
def create_tweets(length):

  import sys
  output = ""
  start = np.random.randint(0, len(dataX)-1)
  pattern = dataX[start]
  for x in pattern:
    output = output + str(int_to_char[x])

  # generate characters
  for i in range(length):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    output = output + str(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

  return output

###**Testing the function**

In [198]:
created = create_tweets(100)

In [199]:
print(created)

l cant believe you have to move i will be over after work to help paint aww rip your walllllll justanotherjerk i wanna c quotno doubtquot soooo to sore to sork to see to sore to sork to see to sore to sork to see to sore to sork to see to sore 


#Generated Text Evaluation

In [197]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
  
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu